# UTSA CS 3793: Assignment-1

**Ramos - Kendall - (yjk504)**






## Learning Objectives


*   Read data from a file and Create a graph
*   Implement Uninformed & Informed searching strategies
*   Apply different searching strategies for the given problem
*   Analyze and Compare the searching strategies


In [260]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Description

This assignment is focused on **python file reading, graph creation** and implementation of **search algorithms**.
In the following sections, you will complete a series of tasks for a made up problem of *Coronavirus in Texas*.

*   Coronavirus is non-discriminatory, in the sense that it can spread from one city to any other city. The only goal of the virus is to spread to all cities in Texas. Find a possible way for the virus to spread (Uninformed Search).
*   To counter the effect of the virus, vaccine needs to be distributed to all cities. One city has more demand than supply, whereas one city has a shortage of vaccines. The goal is to find an **optimal** strategy to transport the vaccine (Informed Search) from the city with high supply (low demand) to the city with low supply (high demand).

The base structure and comments are provided on what should be done. You can use some libraries that help support you for the successful completion of the assignment. However, you **CANNOT** use a complete library for the search algorithms. You can get pieces of code from online, but please cite the source properly.


#Reading Data Files & Creating a 2D Graph

##(45 points)

In this section, you will write code to read the data files provided, cities.csv and distances.csv, and create a 2D graph consisting of nodes and edges. The goal is to use this graph for the 2 search agents that you will create in the next section.

Provided with this lab, on Blackboard, you will find 2 csv files:

*   **cities.csv** - This file contains a list of coordinates for selected cities in Texas, in the following format:
```
San Antonio,29.4685,-98.5254
```
The above line means that San Antonio is located at the latitude and longitude of 29.4685 N and 98.5254 W respectively. Note that the '-ve' sign denotes 'S' for latitude and 'W' for longitude. While performing calculations you will need to ignore the sign.

*   **distances.csv** - This file contains distance values between two cities of Texas, if a path exists, in the following format:
```
San Antonio,New Braunfels,30.80876734
```
The above line denotes that there should be an edge between *San Antonio* and *New Braunfels* and the weight on that edge, i.e. the distance, is *30.80876734*.

In the code blocks below, handle the logic for the graph. Load the graph data from the give files and display a 2D graph of the given data, with labeled nodes and edges. Create as many functions or code blocks as needed.

##Extra Credit (4 points)

Overlay the 2D graph on an image of the Texas state map.





In [261]:
# Add only your imports here

import csv
import networkx as nx
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

from queue import Queue

import heapq


In [262]:
# Assume that the data files are in the following folder -- THIS WILL BE USED BY THE TA
basePath = "/content/drive/My Drive/Colab Notebooks/Artificial Intelligence/Data/"

In [263]:
# Load data from cities.csv with tab as the delimiter
cities_file_path = basePath + 'cities.csv'
cities_data = {}

with open(cities_file_path, 'r') as cities_file:
    csv_reader = csv.reader(cities_file, delimiter=',')  # Use ',' as the delimiter
    for row in csv_reader:
        if len(row) >= 3:
          city_name, latitude, longitude = [value.strip() for value in row]
          #city_name, latitude, longitude = row[0], row[1], row[2]
          cities_data[city_name] = (float(latitude), float(longitude))


# Load data from distances.csv
distances_file_path = basePath + 'distances.csv'
G = nx.Graph()

with open(distances_file_path, 'r') as distances_file:
    csv_reader = csv.reader(distances_file)
    for row in csv_reader:
      city1, city2, distance = [value.strip() for value in row]
      #city1, city2, distance = row
      G.add_edge(city1, city2, weight=float(distance))





In [ ]:
# Display a 2D graph of the given data.
import numpy as np

# Load the Texas state map image
map_image = Image.open("/content/drive/My Drive/Colab Notebooks/Artificial Intelligence/Texas_map.jpg")

# Create a figure and axis for plotting
fig, ax = plt.subplots(figsize=(17, 17))

# Draw the Texas state map image
ax.imshow(map_image, extent=[-110, -91, 25.3, 37])

# Draw the graph on top of the map (replace with your graph drawing code)
pos = {city: (lon, lat) for city, (lat, lon) in cities_data.items()}

# Draw edges with weights
edge_labels = {(city1, city2): f"{G[city1][city2]['weight']:.2f} mi" for city1, city2 in G.edges()}
nx.draw(G, pos, ax=ax, with_labels=False, node_size=25, font_size=10, node_color='b')  # Adjusted node size and font size

# Define the path and cities visited (you need to replace these with your actual data)
path = ['San Antonio', 'New Braunfels', 'Seguin', 'San Marcos', 'Austin', 'Houston', 'College Station']
cities_visited = ['San Antonio', 'New Braunfels', 'Seguin', 'San Marcos', 'Austin', 'Houston', 'College Station']

# Plot the path on the map
path_coordinates = [cities_data[city] for city in path]
path_lats, path_lons = zip(*path_coordinates)
plt.plot(path_lons, path_lats, marker='o', color='r', label='Path')

# Plot the cities visited on the map
visited_coordinates = [cities_data[city] for city in cities_visited]
visited_lats, visited_lons = zip(*visited_coordinates)
plt.scatter(visited_lons, visited_lats, marker='o', color='b', label='Cities Visited')

# Add city labels for all cities (you can adjust these positions as needed)
for city, (lat, lon) in cities_data.items():
    plt.text(lon, lat, city, fontsize=8, ha='right')

# Customize edge label appearance (blend with background and horizontal)
for (city1, city2), label in edge_labels.items():
    edge_x = [pos[city1][0], pos[city2][0]]
    edge_y = [pos[city1][1], pos[city2][1]]
    ax.text(np.mean(edge_x), np.mean(edge_y), label, fontsize=10, ha='center', va='center', alpha=0.7, bbox=dict(facecolor='none', edgecolor='none'))

# Show the legend
plt.legend()

# Show the plot
plt.show()



#Virus Spread - Uninformed Search Agent

##(40 points)

In this section, you will use the graph created in the previous section and create an *uninformed search* agent that will print the path how the virus will spread to all the provided Texas cities. The first confirmed case of the virus was in **Three Rivers** and starts spreading from there. The virus does not discriminate and it needs to spread to all the cities of Texas.

In the following code block, write code to implement **any** uninformed search strategy. You are free to create more code blocks if needed. As the output, print

*   The path or sequence of cities that will be infected by the spread of Coronavirus.
*   The distance travelled by the selected virus spreading strategy.

##Extra Credit (3 points)
On the 2D graph and the Texas state map, overlay the selected path along with the cities visited.

In [ ]:
# Implement ANY uninformed search strategy for the spread of coronavirus from the starting city of 'Three Rivers'

def virus_spread(graph, start_city):
    visited = set()
    spread_path = []
    distance_travelled = 0

    q = Queue()
    q.put(start_city)

    while not q.empty():
        current_city = q.get()
        if current_city not in visited:
            visited.add(current_city)
            spread_path.append(current_city)
            for neighbor in graph.neighbors(current_city):
                if neighbor not in visited:
                    q.put(neighbor)
                    distance_travelled += graph[current_city][neighbor]['weight']

    return spread_path, distance_travelled

# Start the virus spread from 'Three Rivers'
start_city = 'Three Rivers'
spread_path, distance_travelled = virus_spread(G, start_city)

# Print the spread path and distance
print("Virus Spread Path:")
print(spread_path)
print("Distance Travelled:", distance_travelled)


#Vaccine Transportation - Informed Search Agent

##(40 points)

In this section, you will create an *informed search* agent that will be used to transport the vaccine. The city of **San Antonio** has more supply of vaccine than the demand. The goal is to create an **optimal strategy** to transport the vaccine and make it available at the highly affected city of **College Station**, where there is a shortage of vaccines.

In the following code block, write code to implement an **optimal** informed search strategy. You are free to create more code blocks if needed. As the output, print

*   The path / sequence of cities that will be visited in the optimal vaccine transportation strategy.
*   The total distance travelled in the optimal vaccine transportation strategy.


##Extra Credit (3 points)
On the 2D graph and the Texas state map, overlay the selected path along with the cities visited.

In [ ]:
# Implement an OPTIMAL informed search strategy for distributing the vaccine from 'San Antonio' to 'College Station'

# Heuristic function (Euclidean distance between two cities)
def heuristic(city, target_city, cities_data):
    lat1, lon1 = cities_data[city]
    lat2, lon2 = cities_data[target_city]
    return ((lat1 - lat2) ** 2 + (lon1 - lon2) ** 2) ** 0.5

# A* search algorithm for optimal vaccine transportation
def optimal_vaccine_transport(graph, start_city, target_city, cities_data):
    if start_city not in graph.nodes:
        raise ValueError(f"Start city '{start_city}' not found in the graph.")
    if target_city not in graph.nodes:
        raise ValueError(f"Target city '{target_city}' not found in the graph.")

    visited = set()
    transport_path = []
    total_distance = 0

    priority_queue = [(0, start_city)]

    while priority_queue:
        distance, current_city = heapq.heappop(priority_queue)
        if current_city not in visited:
            visited.add(current_city)
            transport_path.append(current_city)

            if current_city == target_city:
                break

            neighbors = list(graph.neighbors(current_city))
            neighbors.sort(key=lambda neighbor: distance + heuristic(neighbor, target_city, cities_data) + graph[current_city][neighbor]['weight'])

            for neighbor in neighbors:
                if neighbor not in visited:
                    heapq.heappush(priority_queue, (distance + graph[current_city][neighbor]['weight'] + heuristic(neighbor, target_city, cities_data), neighbor))

    for i in range(len(transport_path) - 1):
        total_distance += graph[transport_path[i]][transport_path[i + 1]]['weight']

    return transport_path, total_distance



#Submission Instructions

1.   Complete all tasks above - **File MUST contain the output for ALL cells**
2.   Export this notebook as .ipynb
      (File > Download as ipynb)
3.   Upload the .ipynb file on Blackboard

##Rubric

*   (45 points) Reading Data files & Creating a 2D Graph
*   (40 points) Virus Spread - Uninformed Search Agent
*   (40 points) Vaccine Transportation - Informed Search Agent
*   (10 points) Extra Credit - on the Texas map image, overlay the 2D graph and the paths selected by the search agents



